### Intel加速

In [1]:
from sklearnex import patch_sklearn  # Intel加速
patch_sklearn()
import warnings
warnings.filterwarnings('ignore')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import tools
import matplotlib.pyplot as plt
import numpy as np

In [3]:
fileName = '最新训练数据.csv'
y_name = 'if_gxbts'

In [4]:
# 函数起始
import pandas as pd
data_train_origin = pd.read_csv(fileName, encoding='gbk')
#data_test_origin = pd.read_csv('df_lw_test.csv', encoding='gbk')
data = data_train_origin.copy()

In [5]:
# 高空值列直接删除
threshold = 0.7
dataDetect = tools.DataDetect()
high_missing_features = dataDetect.missing_values_table(data, threshold * 100)
data = data.drop(high_missing_features, axis=1)
print('高空值列已经删除. 删除阈值为缺失率大于 {}% '.format(threshold*100),'\n当前列数为 {}'.format(len(data.columns)))

含空值的列有 162 列.
有 21 列的空值阈值超过了 > 70.0%.
空值大于阈值的特征有: ['IS_RETURNMONEY', 'IS_CALL_TELECOM', 'TOTAL_SCORE', 'DIS_DETAIL', 'DIS_TYPE', 'duanduan_time', 'RETURNMONEY', 'IS_SEND_TELECOM', 'DUTY_TYPE', 'CLIENT_MAN', 'RECALL_WAY', 'REAL_RETURNMONEY', 'SERVICE', 'IS_REASON', 'GIS_STATION_CELL', 'APPEAL_DATE', 'discontented_reason', 'offen_5g_rest_cell_top3', 'offen_5g_work_cell_top3', 'offen_5g_cell_top3', 'offen_5g_rest_cell_top2']
高空值列已经删除. 删除阈值为缺失率大于 70.0%  
当前列数为 141


In [6]:
# 目标列及其他特征列初步探索
print('y值统计:{}\n'.format(data[y_name].value_counts()))

singleValueCol = []
multiValueCol = []
numerousValueCol = []
numerousThreshold = 10
for col in data.columns:
    if len(data[col].unique()) == 1: singleValueCol.append(col)
    elif len (data[col].unique()) > numerousThreshold: numerousValueCol.append(col)
    else: multiValueCol.append(col)
print('单值特征:{}\n'.format(len(singleValueCol)))
print('多值特征:{}\n'.format(len(multiValueCol)))
print('超值特征:{}\n'.format(len(numerousValueCol)))

# 卸载单值,探索超值,转换多值
data = data.drop(singleValueCol, axis=1)
print(multiValueCol)

y值统计:if_gxbts
0    42653
1     2506
Name: count, dtype: int64

单值特征:6

多值特征:52

超值特征:83

['IF_JG', 'YY_TS_UP', 'COM_SOURCE', 'RECALL_TIMES', 'IS_REPEAT', 'rt_sa_stitch', 'is_main_net', 'avg3_lan_cs_speed', 'xxzx_label49', 'xxzx_label162', 'bb_zc_flag', 'aqiyi_member', 'is_txhy', 'youku_member', 'is_single', 'is_act3_kd', 'itv_type_name', 'call_out_1m_num', 'flux_use_type', 'speed1000_flag', 'is_home_stu', 'last3_limit_spee', 'dt_tq_fans', 'xxzx_label161', 'adsl_zc_reason', 'prd_age', 'prd_sex', 'xxzx_label274', 'xxzx_label275', 'xxzx_label278', 'c622', 'c60', 'xxzx_label333', 'owe_flag', 'two_kd_flag', 'xxzx_label377', 'fwzj_flag', 'anchor_flag', 'jz_design_flag', 'zwry_flag', 'gwy_flag', 'kdy_flag', 'ywry_flag', 'wmy_flag', 'teach_flag', 'wysj_flag', 'wlxs_flag', 'tszyh_flag', 'skzj_flag', 'is_gxb', 'gxb_wh_ts', 'if_gxbts']


In [7]:
data.columns

Index(['BUSI_PHONE', 'SHEET_NO', 'COM_TYPE', 'DEAL_TIME', 'COM_CONTENT',
       'IF_JG', 'YY_TS_UP', 'TS_TYPE', 'TS_QW', 'LABEL',
       ...
       'offen_4g_rest_cell_top2', 'offen_4g_rest_cell_top3',
       'offen_5g_cell_top1', 'offen_5g_cell_top2', 'offen_5g_work_cell_top1',
       'offen_5g_work_cell_top2', 'offen_5g_rest_cell_top1', 'days_after',
       'time_interval', 'if_gxbts'],
      dtype='object', length=135)

In [26]:
# 将所有object类别转化为float或者int
def dataProcess_convert(data):
    from sklearn.preprocessing import LabelEncoder

    '''
    以下为修改后的代码
    '''

    
    return data
data = dataProcess_convert(data)

In [28]:
# 填充空值
def dataProcess_fillNa(data, if_testData=0, imputerModel = None):
    tFillNa = tools.FillNa()
    imputer = None
    data, imputer = tFillNa.fillNa_fancyimpute(data=data, if_testData=if_testData, imputerModel=imputerModel, iter=100)
    data = tFillNa.fillNa_simpleImputer(data=data, mode='most_frequent')
    return data, imputer
data, imputer = dataProcess_fillNa(data,  if_testData=0, imputerModel = None)
# 请注意，如果fillNa_fancyimpute使用默认配置(即使用MICE),则这里的imputer在后续的对datatest的处理中会再次用到.

In [29]:
# 查询是否全部空值填充完毕
null_counts = data.isnull().sum()
non_zero_nulls = null_counts[null_counts > 0]
non_zero_nulls

Series([], dtype: int64)

In [30]:
# 把y单列，便于后续的特征增加和标准化
def dataProcess_ydiv(data):
    features_col = data.drop('if_lw', axis=1)
    y_col = data['if_lw']
    return features_col, y_col
features_col, y_col = dataProcess_ydiv(data)

In [31]:
# 特征扩展  
def dataProcess_featureExpand(features_col, if_testData = 0, feature_names = None):
    fe = tools.FeatureExpansion()
    if if_testData == 0:
        feature_matrix, feature_names = fe.extend(features_col, debugmode=1) # debug模式参数默认不填写，此处是为了加速调试
    else:
        feature_matrix = fe.extend(features_col, if_testData=if_testData, trained_feature = feature_names)
    return feature_matrix, feature_names

# 注意此处的feature_names会在对测试集做拓展的时候用到，以便数据拓展的时候做同等操作
features_col, feature_names = dataProcess_featureExpand(features_col)

c:\Users\xyue\AppData\Local\Programs\Python\Python311\Lib\site-packages\featuretools\entityset\entityset.py:1910: UserWarning: index id not found in dataframe, creating new integer column
  warnings.warn(
c:\Users\xyue\AppData\Local\Programs\Python\Python311\Lib\site-packages\featuretools\synthesis\deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(
c:\Users\xyue\AppData\Local\Programs\Python\Python311\Lib\site-packages\featuretools\synthesis\dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['sum']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveW

Built 4096 features
Elapsed: 00:17 | Progress: 100%|██████████


In [32]:
# 将特征标准化
def dataProcess_standardScaler(features_col):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    data = pd.DataFrame(StandardScaler().fit_transform(features_col), columns=features_col.columns)
    return data
features_col = dataProcess_standardScaler(features_col)

In [33]:
# 将标准化后的特征矩阵和y拼合回去
def dataProcess_concat(features_col, y_col, y_name):
    data = pd.concat([features_col, pd.DataFrame(y_col, columns=[y_name])], axis=1)
    return data
data = dataProcess_concat(features_col, y_col, y_name)

In [34]:
# 拼合训练数据和测试数据
def data_div(data, tailNum):
    data_test = data.tail(tailNum).reset_index(drop=True)
    data_train = data.head(len(data) - tailNum)
    return data_train, data_test
data_train, data_test = data_div(data, 22000)

In [35]:
def dataProcess_importantceFactor_beforeModel(data, y_name, threshold):
    ifCal = tools.importantceCalculate()
    # 请注意此处可以添加多种重要性判断方法，最后进行过滤
    
    # corrs方法中,默认阈值0.01,即此处threshold如果要改,则应大于0.01.
    corrs = ifCal.featureImportance_corrs(data, y_name, threshold, 'pearson')
    corrs = corrs.sort_values(by=1, ascending=False)
    print(corrs)
    
    # 直接过滤出最后结果
    print('共选择了{}个特征'.format(len(corrs)))
    print('过滤出来的特征为:\n{}'.format(corrs[0].tolist()))
    return corrs
corrs = dataProcess_importantceFactor_beforeModel(data_train, y_name, 0.01)

支持的参数有: pearson,spearman


c:\Users\xyue\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\xyue\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


                                 0         1
1377                         if_lw  1.000000
264   if_ddm_wy / dis_cnts_install  0.112303
985    tczf_score_model / ydwl_myd  0.099222
520           if_ywkf / if_ywkf_cj  0.097924
557           if_ywkf_cj / if_ywkf  0.097924
...                            ...       ...
703               if_znzw / if_wkc  0.010071
1094  tnet_octets_last2 / cj_ratio  0.010044
1271             yyt_num / bmy_num  0.010038
1108  tnet_octets_last2 / ydwl_myd  0.010024
1141     voice_satu_avg / speed_id  0.010002

[1378 rows x 2 columns]
共选择了1378个特征
过滤出来的特征为:
['if_lw', 'if_ddm_wy / dis_cnts_install', 'tczf_score_model / ydwl_myd', 'if_ywkf / if_ywkf_cj', 'if_ywkf_cj / if_ywkf', 'if_ywkf_cj', 'wh_myd / if_ywkf_cj', 'if_ywkf_cj / wh_myd', 'if_ywkf_cj / yyt_myd', 'yyt_myd / if_ywkf_cj', 'wlzl_score_model / if_ywkf_cj', 'kdwl_myd / if_ywkf_cj', 'if_ywkf_cj / ydwl_myd', 'zfgz_myd / if_ywkf_cj', 'if_ywkf_cj / kdwl_myd', 'if_ywkf_cj / wlzl_score_model', 'ydwl_myd / if_ywk

In [36]:
'''
选择特征！！！
根据特征重要性来选择特征，注意此步骤极其重要。
不同的阈值可以筛选出来不同数量的特征,过多过少的特征都会导致模型性能不佳。选择一个恰当的特征数量是极其重要的！！！
'''
def dataProcess_featureSelect(corrs, threshold):
    df_sorted = corrs.sort_values(by=1, ascending=False)
    df_featureSelected = df_sorted[df_sorted[1]>threshold]
    print('共选择了{}个特征\n'.format(len(df_featureSelected)),df_featureSelected)

    return df_featureSelected
df_featureSelected = dataProcess_featureSelect(corrs, 0.03) # 0.01 - 0.05 0.005


共选择了473个特征
                                  0         1
1377                         if_lw  1.000000
264   if_ddm_wy / dis_cnts_install  0.112303
985    tczf_score_model / ydwl_myd  0.099222
520           if_ywkf / if_ywkf_cj  0.097924
557           if_ywkf_cj / if_ywkf  0.097924
...                            ...       ...
782    kdwl_myd / wlzl_score_model  0.030355
281       if_ddm_wy / silent_month  0.030341
498       if_yd / prim_chrge_level  0.030323
1161             wh_myd / speed_id  0.030156
795    last_flux_zs / silent_month  0.030021

[473 rows x 2 columns]


In [37]:
# 根据选择的特征,更新data,以便于后续建模处理,这一步之前不可删除 y_name列
def dataProcess_featureSelectToData(data, df_featureSelected):
    dataSelected = data[df_featureSelected[0]]
    return dataSelected 
dataSelected = dataProcess_featureSelectToData(data_train, df_featureSelected)

In [38]:
null_counts = dataSelected.isnull().sum()
non_zero_nulls = null_counts[null_counts > 0]
non_zero_nulls

Series([], dtype: int64)

In [39]:
# 数据超采样
def sample(X_train, y_train):
    sf = tools.SampleFunction()
    X_resampled, y_resampled = sf.oversample_SMOTE(X_train, y_train)
    return X_resampled, y_resampled

# 数据分割
def model_dataPrepare(data, y_name):
    from sklearn.model_selection import train_test_split
    y = data[y_name]
    X = data.drop(y_name, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=10,stratify=y)
    #X_train, y_train = sample(X_train, y_train)
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = model_dataPrepare(dataSelected, y_name)

In [40]:
def model_train(X_train, y_train):
    from sklearn.ensemble import RandomForestClassifier
    import xgboost as xgb
    import numpy as np
    model = RandomForestClassifier(random_state=12,n_estimators = 180,class_weight={0:1,1:3},min_samples_split=15, min_samples_leaf=8,
                                max_depth=10,n_jobs = -1)
    # 朴素贝叶斯、随机森林、xgboost、LightGBM、CatBoost 
    #model = xgb.XGBClassifier(n_estimators=100,max_depth= 5,learning_rate =0.15, subsample= 0.8,colsample_bytree= 0.8,reg_alpha=0.5)
    model.fit(X_train, y_train)
    return model
model = model_train(X_train, y_train)

In [41]:
def dataProcess_importantceFactor_afterModel(data, y_name):
    ifCal = tools.importantceCalculate()
    # 请注意此处可以添加多种重要性判断方法，最后进行过滤
    corrs = ifCal.featureImportance_corrs(data, y_name, 0.01, 'pearson')

    # 直接过滤出最后结果
    print('过滤出来的特征为:\n{}'.format(corrs[0].tolist()))
    data = data[corrs[0]]
    return data
#datatest = dataProcess_importantceFactor_afterModel(data, y_name)

In [42]:
def model_evaluate(model, X_test, y_test):
    import sklearn.metrics as metrics
    import numpy as np
    pre_test = model.predict(X_test)
    print(metrics.confusion_matrix(y_test, pre_test, labels=[0, 1]))  # 混淆矩阵
    print(metrics.classification_report(y_test, pre_test))
    y_score = model.predict_proba(X_test)
    # 初始化衡量指标，准备遍历求解
    best_F1 = 0
    best_AUC = 0
    best_threshold = 0
    for i in range(1, 20):
        threshold = 0.05 * i
        print('\n当前阈值为:{}\n'.format(threshold))
        pre_test = np.where(y_score[:,1]>threshold,1,0)
        print(metrics.confusion_matrix(y_test, pre_test, labels=[0, 1]))  # 混淆矩阵   估计派单量，覆盖率达到多少,调整阈值如0.2
        print(metrics.classification_report(y_test, pre_test))
        AUC = metrics.roc_auc_score(y_test, pre_test)
        print('AUC: {:.4f}'.format(AUC)) ##ROC值
        F1 = metrics.f1_score(y_test, pre_test,average='macro')
        print('F1-score: {:.4f}'.format(F1)) ## f1值
        
        if F1 > best_F1 and AUC > best_AUC: 
            best_F1 = F1; best_AUC = AUC; best_threshold = threshold
        elif F1 > best_F1 and abs(AUC - best_AUC) < 0.1:
            best_F1 = F1; best_AUC = AUC; best_threshold = threshold
        else:pass
    return best_threshold, best_F1, best_AUC
model_evaluate(model, X_test, y_test)

[[9808  192]
 [ 306  694]]
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98     10000
         1.0       0.78      0.69      0.74      1000

    accuracy                           0.95     11000
   macro avg       0.88      0.84      0.86     11000
weighted avg       0.95      0.95      0.95     11000


当前阈值为:0.05

[[3635 6365]
 [   1  999]]
              precision    recall  f1-score   support

         0.0       1.00      0.36      0.53     10000
         1.0       0.14      1.00      0.24      1000

    accuracy                           0.42     11000
   macro avg       0.57      0.68      0.39     11000
weighted avg       0.92      0.42      0.51     11000

AUC: 0.6813
F1-score: 0.3860

当前阈值为:0.1

[[5261 4739]
 [   5  995]]
              precision    recall  f1-score   support

         0.0       1.00      0.53      0.69     10000
         1.0       0.17      0.99      0.30      1000

    accuracy                           0.57    

(0.55, 0.8585103596476222, 0.82255)

In [43]:
data_test_out = data_test.copy()

In [44]:
# 开始对测试集进行处理
def dataProcess_testData(data_test_out,data_test_origin, df_featureSelected, y_name):
    import numpy as np
    dataSelected = dataProcess_featureSelectToData(data_test_out, df_featureSelected)
    dataSelected = dataSelected.drop(y_name, axis=1)
    y_score = model.predict_proba(dataSelected)
    df = pd.DataFrame({"lables": list(data_test_origin['id']), "scores": list(y_score)})
    for i in range(1, 20):
        threshold = 0.05 * i
        df['label_{:.2f}'.format(threshold)] = np.where(y_score[:,1]>threshold,1,0)
        print(df['label_{:.2f}'.format(threshold)].value_counts(), '\n')
    for col in df.columns[2:]:
        print('\n',df[col].value_counts())
    df.to_csv('outputResult.csv')
dataProcess_testData(data_test_out,data_test_origin, df_featureSelected, y_name)

label_0.05
1    17085
0     4915
Name: count, dtype: int64 

label_0.10
1    13482
0     8518
Name: count, dtype: int64 

label_0.15
0    11812
1    10188
Name: count, dtype: int64 

label_0.20
0    14959
1     7041
Name: count, dtype: int64 

label_0.25
0    17318
1     4682
Name: count, dtype: int64 

label_0.30
0    18781
1     3219
Name: count, dtype: int64 

label_0.35
0    19773
1     2227
Name: count, dtype: int64 

label_0.40
0    20416
1     1584
Name: count, dtype: int64 

label_0.45
0    20816
1     1184
Name: count, dtype: int64 

label_0.50
0    21116
1      884
Name: count, dtype: int64 

label_0.55
0    21313
1      687
Name: count, dtype: int64 

label_0.60
0    21519
1      481
Name: count, dtype: int64 

label_0.65
0    21684
1      316
Name: count, dtype: int64 

label_0.70
0    21784
1      216
Name: count, dtype: int64 

label_0.75
0    21884
1      116
Name: count, dtype: int64 

label_0.80
0    21956
1       44
Name: count, dtype: int64 

label_0.85
0    21987
1 